In [4]:
from sympy import *
x = Symbol("x")
m = 1
y = x**2
dx_dt = Symbol("dx_dt")
dy_dt = diff(y, x) * dx_dt
v = simplify((dx_dt ** 2 + dy_dt ** 2) ** 0.5)

T = simplify(0.5 * m * v ** 2)
V = m * y
L = T - V

ddx_dtt = simplify(sympy.diff(L, x))


NameError: name 'sympy' is not defined

In [80]:
# Pendulum on movable support

m = Symbol('m')
l = Symbol('l')
M = Symbol('M')
g = Symbol('g')

x = Symbol('x')
theta = Symbol('theta')
x_dot = Symbol('x_dot')
theta_dot = Symbol('theta_dot')

x_pendulum = x + l * sin(theta)
y_pendulum = - l * cos(theta)

x_pendulum_dot = diff(x_pendulum, x) * x_dot + diff(x_pendulum, theta) * theta_dot
y_pendulum_dot = diff(y_pendulum, x) * x_dot + diff(y_pendulum, theta) * theta_dot

V = y_pendulum * m * g
T = 0.5 * M * x_dot ** 2 + 0.5 * m * (x_pendulum_dot ** 2 + y_pendulum_dot ** 2)

L = T - V

x_dot_dot = simplify(diff(L, x))
theta_dot_dot = simplify(diff(L, theta))

In [81]:
[theta_dot_dot]

[-l*m*(g + theta_dot*x_dot)*sin(theta)]

In [50]:
double_pendulum = {
    'particles': [
        { 'name': 'A', 
          'mass': 1,
          'position': { 'type': 'pole', 'origin': {'type': 'fixed', 'position': (0, 0) }, 'length': 1 } 
        },
        { 'name': 'B',
          'mass': 1, 
          'position': { 'type': 'pole', 'origin': {'type': 'particle', 'particle': 'A' }, 'length': 1 } 
        }
    ],
    'gravity': True
}

pendulum_on_moving_support = {
    'particles': [
        { 'name': 'A',
          'position': { 'type': 'constrained', 'x': 'a', 'y': '0' } },
        { 'name': 'B',
          'position': { 'type': 'pole', 'origin': {'type': 'particle', 'particle': 'A' }, 'length': 1 } },
    ],
    'gravity': True
}

double_parabola = {
    'particles': [
        { name: 'A', position: { 'type': 'constrained', 'x': '-a**2 - 1', 'y': 'a' } },
        { name: 'B', position: { 'type': 'constrained', 'x': 'a**2 + 1', 'y': 'a' } },
    ]
    'gravity': False,
    'springs': [
        { 'from': 'A', 'to': 'B', 'k': 1, 'length': 2 }
    ]
}

# to add later: charge, springs to fixed points

SyntaxError: invalid syntax (<ipython-input-50-cb403136b407>, line 30)

In [75]:
from sympy.parsing.sympy_parser import parse_expr
                                        
class Particle:
    def __init__(self, pojo):
        self.name = pojo["name"]
        self.position = pojo["position"]
        self.mass = pojo['mass']
        self.position_type = pojo["position"]["type"]
        self.parameters = self.build_parameters()
    
    def position_parameters(self):
        if self.position_type == 'free':
            return { 'x': Symbol(self.name + "_x"), 'y': Symbol(self.name + "_y") }
        elif self.position_type == 'constrained':
            return { 'a': Symbol(self.name + "_a") }
        elif self.position_type == 'pole':
            return { 'theta': Symbol(self.name + "_theta") }
    
    def build_parameters(self):
        params = self.position_parameters()
        return {**params, **{ x + "_dot": Symbol(y.name + "_dot") for (x, y) in params.items() }}
    
    def cartesian_position(self, system):
        if self.position_type == 'free':
            return [self.parameters['x'], self.parameters['y']]
        if self.position_type == "constrained":
            x = parse_expr(self.position['x']).subs('a', self.parameters['a'])
            y = parse_expr(self.position['y']).subs('a', self.parameters['a'])
            return [x, y]
        if self.position_type == 'pole':
            if self.position['origin']['type'] == 'fixed':
                origin_x, origin_y = self.position['origin']['position']
            elif self.position['origin']['type'] == 'particle':
                origin_x, origin_y = system.cartesian_position(self.position['origin']['particle'])
                
            theta = self.parameters['theta']
            length = self.position['length']
            x = sin(theta) * length + origin_x
            y = cos(theta) * length + origin_y
            return [x, y]
        
    def cartesian_velocity(self, system):
        if self.position_type == 'free':
            return [self.parameters['x_dot'], self.parameters['y_dot']]
        if self.position_type == "constrained":
            x = parse_expr(self.position['x']).subs('a', self.parameters['a'])
            a = self.parameters['a']
            dx_da = diff(x, a)
            x_dot = dx_da * self.parameters['a_dot']
            
            y = parse_expr(self.position['y']).subs('a', self.parameters['a'])
            dy_da = diff(y, a)
            y_dot = dy_da * self.parameters['a_dot']
            
            return [x_dot, y_dot]
        if self.position_type == 'pole':
            if self.position['origin']['type'] == 'fixed':
                origin_x_dot, origin_y_dot = 0
            elif self.position['origin']['type'] == 'particle':
                origin_x_dot, origin_y_dot = system.cartesian_velocity(self.position['origin']['particle'])
                
            theta = self.parameters['theta']
            length = self.position['length']
            x_dot = cos(theta_dot) * length + origin_x_dot
            y_dot = cos(theta_dot) * length + origin_y_dot
            return [x_dot, y_dot]
        
    def potential_energy(self, system):
        PE = 0
        if system.gravity:
            PE += self.mass * 9.8 * self.cartesian_position(system)[1]
        return PE
        
    def kinetic_energy(self, system):
        return 0.5 * self.mass * sum(z**2 for z in self.cartesian_velocity(system))
    
    
class System:
    def __init__(self, pojo):
        self.particles = { x['name']: Particle(x) for x in pojo['particles'] }
        self.gravity = pojo['gravity']
    
    def cartesian_position(name):
        self.particles[name].cartesian_position(self)
        
    def kinetic_energy(self):
        return sum(particle.kinetic_energy(self) for particle in self.particles.values())
    
    def potential_energy(self):
        return sum(particle.potential_energy(self) for particle in self.particles.values())
    
    def lagrangian(self):
        return self.kinetic_energy() - self.potential_energy()

    def equations_of_motion(self):
        lagrangian = self.lagrangian()
        return {
            param : diff(lagrangian, param) 
                for param in particle.position_parameters().values()
                for particle in self.particles.values()
        }
    
        
    
# def equations_of_motion(system):


In [47]:
particle1_pojo = { 'name': 'A', 
          'mass': 1,
          'position': { 'type': 'constrained', 'x': '-a**2 - 1', 'y': 'a' }
        }

particle = Particle(particle1_pojo)

In [76]:
simple_system_pojo = {
    'particles': [
        {
            'name': 'A',
            'mass': 1,
            'position': { 'type': 'constrained', 'x': 'a', 'y': 'a' }
        }
    ],
    'gravity': True
}

simple_system = System(simple_system_pojo)
simple_system.equations_of_motion()

{A_a: -9.80000000000000}